In [1]:
% pylab inline

%load_ext autoreload
%autoreload 2
import os
import time
import csv
import warnings

import requests
import urllib.request
import bs4
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm

import indeed_scraping

warnings.filterwarnings("ignore")

%config InlineBackend.figure_format = 'retina'

DATA_DIR = os.path.join(os.getcwd(), 'data')
try:
    os.makedirs(DATA_DIR)
except FileExistsError:
    pass

Populating the interactive namespace from numpy and matplotlib


### Filters

In [66]:
query = 'cyber+security'
pick_levels = [
    'entry_level', 'mid_level', 'senior_level'
]

indeed_experience_filter = ['exp_12_24', 'exp_25_60', 'exp_61_120', 'exp_121+']
indeed_degree_filter = [
#     , 'associate','diploma'
    'bachelor', 'doctor', 'master',
]

min_jobs = 5
min_resumes = 25

### Metropolital areas

#### Download data

In [3]:
urllib.request.urlretrieve(
    'https://www2.census.gov/programs-surveys/popest/datasets/2010-2017/metro/totals/cbsa-est2017-alldata.csv', 
    os.path.join(DATA_DIR, 'cbsa-est2017-alldata.csv'))

('/Users/keld/poseidon/data/cbsa-est2017-alldata.csv',
 <http.client.HTTPMessage at 0x10eb879e8>)

Download free version of city map information here https://simplemaps.com/data/us-cities to data folder.

Link will be to: https://simplemaps.com/static/data/us-cities/uscitiesv1.4.csv

In [4]:
assert os.path.exists(os.path.join(DATA_DIR, 'uscitiesv1.4.csv')), 'follow steps above'

You are ready to proceed

#### Join metropolitan area data

In [5]:
df_us_cities = pd.read_csv(os.path.join(DATA_DIR, 'uscitiesv1.4.csv'))
df_us_cities['county'] = df_us_cities['county_name']

In [6]:
df_us_cities = df_us_cities[['state_id', 'city', 'county', 'population_proper', 'lat', 'lng']].fillna(0)

In [7]:
df_us_cities.head()

,state_id,city,county,population_proper,lat,lng
0,WA,Prairie Ridge,Pierce,0.0,47.1443,-122.1408
1,WA,Edison,Skagit,0.0,48.5602,-122.4311
2,WA,Packwood,Lewis,0.0,46.6085,-121.6702
3,WA,Wautauga Beach,Kitsap,0.0,47.5862,-122.5482
4,WA,Harper,Kitsap,0.0,47.5207,-122.5196


In [8]:
df_cbsa = pd.read_csv(os.path.join(DATA_DIR, 'cbsa-est2017-alldata.csv'), encoding = "latin1")

In [9]:
df_cbsa[['CBSA', 'NAME', 'STCOU']].columns

Index(['CBSA', 'NAME', 'STCOU'], dtype='object')

In [10]:
print('Unique number of CBSA:', len(df_cbsa['CBSA'].unique()))

Unique number of CBSA: 933


In [11]:
df_cbsa = df_cbsa[['CBSA', 'NAME', 'STCOU']]
df_cbsa['county'] = df_cbsa['NAME'].apply(lambda x: x.split(', ')[0].replace(' County', ''))
df_cbsa['state_id'] = df_cbsa['NAME'].apply(lambda x: x.split(', ')[1])

In [12]:
df_cbsa.head()

,CBSA,NAME,STCOU,county,state_id
0,10180,"Abilene, TX",NaN,Abilene,TX
1,10180,"Callahan County, TX",48059.0,Callahan,TX
2,10180,"Jones County, TX",48253.0,Jones,TX
3,10180,"Taylor County, TX",48441.0,Taylor,TX
4,10420,"Akron, OH",NaN,Akron,OH


In [13]:
df_areas = df_cbsa.join(df_us_cities.set_index(['county', 'state_id']), on=['county', 'state_id'], how='inner')
df_areas = df_areas

In [14]:
df_areas.head()[['CBSA', 'county', 'state_id', 'city', 'population_proper', 'lat', 'lng']]

,CBSA,county,state_id,city,population_proper,lat,lng
1,10180,Callahan,TX,Rowden,0.0,32.2040,-99.3440
1,10180,Callahan,TX,Cross Plains,986.0,32.1271,-99.1658
1,10180,Callahan,TX,Clyde,3842.0,32.4056,-99.5039
1,10180,Callahan,TX,Baird,1513.0,32.3960,-99.3962
1,10180,Callahan,TX,Putnam,95.0,32.3703,-99.1955


In [15]:
df_areas = df_areas.reset_index(drop=True)

In [16]:
df_areas.head()

,CBSA,NAME,STCOU,county,state_id,city,population_proper,lat,lng
0,10180,"Callahan County, TX",48059.0,Callahan,TX,Rowden,0.0,32.2040,-99.3440
1,10180,"Callahan County, TX",48059.0,Callahan,TX,Cross Plains,986.0,32.1271,-99.1658
2,10180,"Callahan County, TX",48059.0,Callahan,TX,Clyde,3842.0,32.4056,-99.5039
3,10180,"Callahan County, TX",48059.0,Callahan,TX,Baird,1513.0,32.3960,-99.3962
4,10180,"Callahan County, TX",48059.0,Callahan,TX,Putnam,95.0,32.3703,-99.1955


### demand

In [17]:
df_demand = pd.read_csv(os.path.join(DATA_DIR, 'cyber+security_all_partitions.tsv'), sep='\t')
df_demand = df_demand[['company_name', 'location', 'partition', 'title']]
df_demand['location'] = df_demand['location'].str.strip()

In [18]:
df_demand['level'] = df_demand['partition'].apply(lambda x: x.split('-')[1])
df_demand['salary_est'] = df_demand['partition'].apply(lambda x: x.split('-')[1])

In [19]:
df_demand = df_demand[df_demand['level'].isin(pick_levels)]

In [20]:
df_demand['level'].unique()

array(['senior_level', 'mid_level'], dtype=object)

In [21]:
len(df_demand)

15594

In [22]:
df_demand = df_demand[df_demand['location'].apply(lambda x: ', ' in x)]
df_demand['city'] = df_demand['location'].apply(lambda x: x.split(', ')[0])
df_demand['state_id'] = df_demand['location'].apply(lambda x: x.split(', ')[1])

In [23]:
df_demand.head()

,company_name,location,partition,title,level,salary_est,city,state_id
0,Jackson-National-Life-Insurance-Company,"Lansing, MI","cyber+security-senior_level-$120,000","AVP, Cybersecurity Response",senior_level,senior_level,Lansing,MI
1,Occidental-Petroleum,"Houston, TX","cyber+security-senior_level-$120,000",IT Cyber Security Advisor,senior_level,senior_level,Houston,TX
2,\n Ingersoll Consulting Inc.,"Washington, DC","cyber+security-senior_level-$120,000",Cyber Security Engineer - Lead,senior_level,senior_level,Washington,DC
3,Saab,"Syracuse, NY","cyber+security-senior_level-$120,000",Senior Staff Systems Engineer; Saab Defense an...,senior_level,senior_level,Syracuse,NY
4,\n Executive Office of Energy and Environme...,"Boston, MA","cyber+security-senior_level-$120,000",Chief Information Security Officer,senior_level,senior_level,Boston,MA


In [24]:
df_demand_area = (
    df_areas
    .reset_index(drop=True)
    .join(
        df_demand.set_index(['city', 'state_id']), 
        on=['city', 'state_id'], 
        how='right')
    .reset_index(drop=True)
)

In [25]:
df_demand_area.head()

,CBSA,NAME,STCOU,county,state_id,city,population_proper,lat,lng,company_name,location,partition,title,level,salary_est
0,10420.0,"Summit County, OH",39153.0,Summit,OH,Akron,197633.0,41.0802,-81.5219,University-of-Akron,"Akron, OH","cyber+security-senior_level-$70,000",Lead Information Security Analyst,senior_level,senior_level
1,10500.0,"Dougherty County, GA",13095.0,Dougherty,GA,Albany,73801.0,31.5776,-84.1762,"Cask,-LLC","Albany, GA","cyber+security-senior_level-$70,000",Information Assurance / Cybersecurity Analyst ...,senior_level,senior_level
2,10540.0,"Linn County, OR",41043.0,Linn,OR,Albany,53211.0,44.6274,-123.0966,\n Legacy Solutions,"Albany, OR","cyber+security-senior_level-$70,000",Microsoft Windows Server Administrator,senior_level,senior_level
3,10580.0,"Albany County, NY",36001.0,Albany,NY,Albany,98111.0,42.6664,-73.7987,Goldman-Sachs,"Albany, NY","cyber+security-senior_level-$90,000",PWM Technology - Ayco Technology Tech Risk Sen...,senior_level,senior_level
4,10740.0,"Bernalillo County, NM",35001.0,Bernalillo,NM,Albuquerque,559277.0,35.1055,-106.6476,SAIC,"Albuquerque, NM","cyber+security-mid_level-$100,000",SMC Cyber Security Lead Job,mid_level,mid_level


In [26]:
len(df_demand)

15310

In [27]:
df_demand_area['jobs'] = 1
df_demand_count = pd.DataFrame(df_demand_area.groupby('CBSA').count()['jobs'].reset_index())

### Supply

In [28]:
df_supply = pd.read_csv(os.path.join(DATA_DIR, 'resumes_cyber+security_all_partitions.tsv'), sep='\t')
df_supply = df_supply[df_supply['location'].apply(lambda x: isinstance(x, str) and ', ' in x)]
df_supply['city'] = df_supply['location'].apply(lambda x: x.split(', ')[0])
df_supply['state_id'] = df_supply['location'].apply(lambda x: x.split(', ')[1])
df_supply = df_supply[['experience', 'company', 'degree', 'partition', 'city', 'state_id']]

In [29]:
df_supply['indeed_experience'] = df_supply['partition'].apply(lambda x: x.split('-')[-2])
df_supply['indeed_degree'] = df_supply['partition'].apply(lambda x: x.split('-')[-1])

In [30]:
df_supply['indeed_degree'].unique()

array(['bachelor', 'master', 'associate', 'doctor', 'diploma'], dtype=object)

In [31]:
df_supply = df_supply[df_supply['indeed_experience'].isin(indeed_experience_filter)]
df_supply = df_supply[df_supply['indeed_degree'].isin(indeed_degree_filter)]

In [32]:
df_supply.head()

,experience,company,degree,partition,city,state_id,indeed_experience,indeed_degree
4800,Senior Principal Cyber Security Engineer,Smiths Medical ASD,Certification,resumes-cyber+security-exp_25_60-master,Hudson,WI,exp_25_60,master
4802,Night Auditor,Even Hotel,MS,resumes-cyber+security-exp_25_60-master,Melbourne,FL,exp_25_60,master
4803,IT Analyst,E2 Labs,Master's,resumes-cyber+security-exp_25_60-master,Manchester,NH,exp_25_60,master
4804,Computer Engineer (0854) GS-13,"US Army, USNORTHCOM",Masters of Science,resumes-cyber+security-exp_25_60-master,Castle Rock,CO,exp_25_60,master
4805,Information System Security Officer,Raytheon Missile Systems,Associate of Applied Science,resumes-cyber+security-exp_25_60-master,Gilbert,AZ,exp_25_60,master


In [33]:
df_supply_area = (
    df_areas
    .reset_index(drop=True)
    .join(
        df_supply.set_index(['city', 'state_id']), 
        on=['city', 'state_id'], 
        how='right')
    .reset_index()
)

In [34]:
# df_supply_area['CBSA'] = df_supply_area['CBSA'].astype(int)

In [35]:
df_supply_area.head()

,index,CBSA,NAME,STCOU,county,state_id,city,population_proper,lat,lng,experience,company,degree,partition,indeed_experience,indeed_degree
0,27,10180.0,"Taylor County, TX",48441.0,Taylor,TX,Abilene,122225.0,32.4543,-99.7384,Programming Tutor,Hardin-Simmons University,Masters of Science,resumes-cyber+security-exp_61_120-master,exp_61_120,master
1,27,10180.0,"Taylor County, TX",48441.0,Taylor,TX,Abilene,122225.0,32.4543,-99.7384,Programming Tutor,Hardin-Simmons University,Masters of Science,resumes-cyber+security-exp_61_120-master,exp_61_120,master
2,27,10180.0,"Taylor County, TX",48441.0,Taylor,TX,Abilene,122225.0,32.4543,-99.7384,Programming Tutor,Hardin-Simmons University,Masters of Science,resumes-cyber+security-exp_61_120-master,exp_61_120,master
3,27,10180.0,"Taylor County, TX",48441.0,Taylor,TX,Abilene,122225.0,32.4543,-99.7384,Programming Tutor,Hardin-Simmons University,Masters of Science,resumes-cyber+security-exp_61_120-master,exp_61_120,master
4,33,10420.0,"Portage County, OH",39133.0,Portage,OH,Kent,30071.0,41.1491,-81.3610,Hadoop Developer,Persistent Systems Limited,Master of Science,resumes-cyber+security-exp_12_24-master,exp_12_24,master


In [36]:
df_supply_area['resumes'] = 1
df_supply_count = pd.DataFrame(df_supply_area.groupby('CBSA').count()[['resumes']].reset_index())

### Affortability

In [37]:
os.listdir(DATA_DIR)
df_relative_cost = pd.read_csv(os.path.join(DATA_DIR, 'cities_relative_cost.tsv'), sep='\t')
df_relative_cost['location'] = df_relative_cost['city']
df_relative_cost = df_relative_cost.drop(columns=['city'], axis=1)[['location', 'relative_cost']]
df_relative_cost = df_relative_cost.reset_index(drop=True)
df_relative_cost = df_relative_cost[df_relative_cost['location'].apply(lambda x: isinstance(x, str) and ', ' in x)]
df_relative_cost['city'] = df_relative_cost['location'].apply(lambda x: x.split(', ')[0])
df_relative_cost['state_id'] = df_relative_cost['location'].apply(lambda x: x.split(', ')[1])


In [38]:
df_relative_cost_area = (
    df_areas
    .reset_index(drop=True)
    .join(
        df_relative_cost.set_index(['city', 'state_id']), 
        on=['city', 'state_id'], 
        how='right')
    .reset_index()
)

In [39]:
df_relative_cost_area['affortability'] = 1 / df_relative_cost_area['relative_cost']
df_relative_cost_area_mean = df_relative_cost_area[['CBSA', 'affortability']].groupby('CBSA').mean()


In [40]:
df_relative_cost_area_mean.head()

,affortability
CBSA,
10180.0,2.180452
10580.0,1.869011
10740.0,2.179520
10900.0,2.033766
11100.0,2.308799


### Poseidon_score

In [41]:
def calc_poseidon_score(df): 
    return (df['resumes'] + df['mean_resumes']) / (df['jobs'] + df['mean_jobs']) * df['affortability']**0.5

#### Pull data together

In [42]:
df_relative_cost_area_mean.head()

,affortability
CBSA,
10180.0,2.180452
10580.0,1.869011
10740.0,2.179520
10900.0,2.033766
11100.0,2.308799


In [43]:
df_demand_supply = (
    df_demand_count
    .reset_index(drop=True)
    .join(
        df_supply_count.set_index('CBSA'), 
        how='inner', 
        on='CBSA'))


In [44]:
df_demand_supply.head()

,CBSA,jobs,resumes
0,10420.0,1,10
3,10580.0,1,43
4,10740.0,13,46
5,10900.0,11,21
7,11100.0,9,7


In [45]:
df_supply_demand_relative_cost = (
    df_demand_supply
    .join(
        df_relative_cost_area_mean, 
        how='inner', 
        on='CBSA'))

In [46]:
df_supply_demand_relative_cost.head()

,CBSA,jobs,resumes,affortability
3,10580.0,1,43,1.869011
4,10740.0,13,46,2.179520
5,10900.0,11,21,2.033766
7,11100.0,9,7,2.308799
8,11460.0,3,39,1.768038


In [47]:
df_supply_demand_relative_cost['mean_jobs'] = df_supply_demand_relative_cost['jobs'].mean()
df_supply_demand_relative_cost['mean_resumes'] = df_supply_demand_relative_cost['resumes'].mean()

#### Calculate Poseidon score

In [48]:
df_poseidon = df_supply_demand_relative_cost
df_poseidon['poseidon_score'] = calc_poseidon_score(df_poseidon)
df_poseidon = df_poseidon.sort_values(by='poseidon_score', ascending=False).reset_index(drop=True)

In [49]:
df_poseidon.to_csv(os.path.join(DATA_DIR, 'posoidon_scores.tsv'), sep='\t')

In [50]:
df_poseidon = df_poseidon[(df_poseidon['jobs'] >= min_jobs) & (df_poseidon['resumes'] >= min_resumes)] 

In [51]:
poseidon_cols = ['CBSA', 'jobs', 'resumes', 'affortability', 'poseidon_score']

In [52]:
df_poseidon[:10][poseidon_cols]

,CBSA,jobs,resumes,affortability,poseidon_score
0,41700.0,29,291,2.199351,6.487989
1,33100.0,36,275,1.602249,4.977234
2,45300.0,35,242,1.844013,4.961113
4,19100.0,155,636,1.947947,4.822687
5,17820.0,35,187,1.914641,4.325588
6,38300.0,27,174,1.734351,4.281115
8,16980.0,120,432,1.786656,4.032228
9,19820.0,48,197,1.977798,4.029019
10,28140.0,9,79,2.073653,4.010206
11,26900.0,11,89,1.854895,3.867925


In [53]:
df_poseidon[-10:][poseidon_cols]

,CBSA,jobs,resumes,affortability,poseidon_score
87,17140.0,45,33,2.070905,2.166226
89,26420.0,249,342,1.623174,1.925491
90,26620.0,106,81,2.259959,1.887136
91,45060.0,80,56,1.963573,1.830731
92,18140.0,127,118,1.953683,1.830356
93,19380.0,113,61,2.248532,1.645573
94,41940.0,193,207,1.201934,1.446402
95,41860.0,251,276,1.244465,1.445579
96,14460.0,427,324,1.413643,1.109330
97,42660.0,460,156,1.556154,0.695397


##### List of cities rated to be good for hiring

In [54]:
df_top_cities = pd.read_csv(os.path.join(DATA_DIR, 'top_cities.csv'))

In [55]:
df_poseidon.join(
    df_top_cities.set_index('CBSA'), on='CBSA', how='right').sort_values(by='poseidon_score', ascending=False).dropna()

,CBSA,jobs,resumes,affortability,mean_jobs,mean_resumes,poseidon_score,city,County,State,state_name
1,33100.0,36.0,275.0,1.602249,69.30303,139.060606,4.977234,Fort Lauderdale,Broward,FL,Florida
1,33100.0,36.0,275.0,1.602249,69.30303,139.060606,4.977234,Doral,Miami-Dade,FL,Florida
6,38300.0,27.0,174.0,1.734351,69.30303,139.060606,4.281115,Pittsburgh,Allegheny,PA,Pennsylvania
10,28140.0,9.0,79.0,2.073653,69.30303,139.060606,4.010206,Kansas City,Wyandotte,KS,Kansas
35,40900.0,12.0,63.0,1.712498,69.30303,139.060606,3.252295,Folsom,Sacramento,CA,California
38,36540.0,16.0,50.0,2.113289,69.30303,139.060606,3.221929,Omaha,Douglas,NE,Nebraska
71,41620.0,20.0,32.0,2.012675,69.30303,139.060606,2.717507,Salt Lake City,Salt Lake,UT,Utah
78,31080.0,190.0,412.0,1.557777,69.30303,139.060606,2.652433,Santa Ana,Orange,CA,California
91,45060.0,80.0,56.0,1.963573,69.30303,139.060606,1.830731,Canastota,Madison,NY,New York


#### Poseidon score of Bay Area

In [56]:
SF_CBSA = ['10500','10580','10540','41860','34980','41940','41180','26900','18140','35620','24860','38300','25540']

In [57]:
df_sf = df_poseidon[df_poseidon['CBSA'].isin(SF_CBSA)]

In [58]:
df_sf['affortability'] = df_sf['affortability'].mean() / len(df_sf['affortability'])

In [59]:
print('Poseidon score of SF: {:.2f}'.format(calc_poseidon_score(df_sf.sum())))

Poseidon score of SF: 2.51


Which is in the bottom quarter!!! 

### Map Poseidon score 

In [60]:
df_poseidon_map = (
    df_poseidon[poseidon_cols].join(
        df_areas.set_index('CBSA'), on='CBSA', how='outer')
    .dropna())

In [61]:
len(df_poseidon_map)

8168

In [62]:
[df_poseidon_map['lat'].min(), df_poseidon_map['lat'].max()]

[25.441800000000001, 48.283700000000003]

In [63]:
df_poseidon_map.head()

,CBSA,jobs,resumes,affortability,poseidon_score,NAME,STCOU,county,state_id,city,population_proper,lat,lng
0,41700.0,29.0,291.0,2.199351,6.487989,"Atascosa County, TX",48013.0,Atascosa,TX,Christine,416.0,28.7863,-98.4977
0,41700.0,29.0,291.0,2.199351,6.487989,"Atascosa County, TX",48013.0,Atascosa,TX,Jourdanton,4327.0,28.9139,-98.5410
0,41700.0,29.0,291.0,2.199351,6.487989,"Atascosa County, TX",48013.0,Atascosa,TX,Campbellton,0.0,28.7475,-98.3025
0,41700.0,29.0,291.0,2.199351,6.487989,"Atascosa County, TX",48013.0,Atascosa,TX,Leming,0.0,29.0684,-98.4722
0,41700.0,29.0,291.0,2.199351,6.487989,"Atascosa County, TX",48013.0,Atascosa,TX,Pleasanton,10393.0,28.9636,-98.4940


In [108]:
from bokeh.io import output_file, output_notebook, show
from bokeh.models import (
  GMapPlot, GMapOptions, ColumnDataSource, Circle, LogColorMapper, BasicTicker, ColorBar,
    DataRange1d, PanTool, WheelZoomTool, BoxSelectTool
)
from bokeh.models.glyphs import Text
from bokeh.models.annotations import Label, Title

from bokeh.models.mappers import ColorMapper, LinearColorMapper
from bokeh.palettes import Viridis5

map_options = GMapOptions(lat=37.88, lng=-102.23, map_type="roadmap", zoom=3)

plot = GMapPlot(
    x_range=DataRange1d(), y_range=DataRange1d(), map_options=map_options
)

Head_text = "Poseidon Scores: " + query

filters = [
    ('Job levels', pick_levels),
    ('Experience levels', indeed_experience_filter),
    ('Degrees', indeed_degree_filter),
    ('Minimum jobs', min_jobs),
    ('Minimum resumes', min_resumes),
]
        
plot.title.text = Head_text              
plot.title.text_font_size = '16pt'
plot.add_layout(Label(text='', **label_opts, text_font_size='12pt'), 'above')
    
# For GMaps to function, Google requires you obtain and enable an API key:
#
#     https://developers.google.com/maps/documentation/javascript/get-api-key
#
# Replace the value below with your personal API key:
plot.api_key = open('google_maps_api_key.txt').read()

color_mapper = LogColorMapper(
    palette="Viridis5", low=df_poseidon_map.poseidon_score.min(), high=df_poseidon_map.poseidon_score.max())
# color_mapper = LinearColorMapper(palette=Viridis5)

source = ColumnDataSource(
    data=dict(
        lat=df_poseidon_map.lat.astype(str).tolist(),
        lon=df_poseidon_map.lng.astype(str).tolist(),
#         size=df_poseidon_map.poseidon_score.tolist(),
        size=[5 for v in df_poseidon_map.poseidon_score.tolist()],
        color=df_poseidon_map.poseidon_score.tolist()
    )
)

label_opts = dict(
    x=0, y=0,
    x_units='screen', y_units='screen'
)

for attribute, values in filters:
    if isinstance(values, list):
        label_text = attribute + ': [' + ', '.join(values) + ']'
    else:
        label_text = attribute + ': ' + str(values)

    plot.add_layout(Label(text=label_text, **label_opts, text_font_size='10pt'), 'below')

circle = Circle(x="lon", y="lat", size="size", 
                fill_color={'field': 'color', 'transform': color_mapper}, fill_alpha=0.5, line_color=None)
plot.add_glyph(source, circle)

color_bar = ColorBar(color_mapper=color_mapper, ticker=BasicTicker(),
                     label_standoff=12, border_line_color=None, location=(0,0))
plot.add_layout(color_bar, 'right')

plot.add_tools(PanTool(), WheelZoomTool(), BoxSelectTool())
plot.toolbar_location="above"
output_file("poseidon_scores_cyber_security_USA.html")

show(plot)

W-1005 (SNAPPED_TOOLBAR_ANNOTATIONS): Snapped toolbars and annotations on the same side MAY overlap visually: GMapPlot(id='443ba221-9349-4210-9504-d741dd1e6b4a', ...)
W-1005 (SNAPPED_TOOLBAR_ANNOTATIONS): Snapped toolbars and annotations on the same side MAY overlap visually: GMapPlot(id='94b563ae-920e-4a61-aa55-99f346a8933e', ...)
E-1001 (BAD_COLUMN_NAME): Glyph refers to nonexistent column name: queries: cyber+securityr>Job levels: [entry_level mid_level senior_level]r>Experience levels: [exp_12_24 exp_25_60 exp_61_120 exp_121+]r>Degrees: [bachelor doctor master]r>Minimum jobs: 5r>Minimum resumes: 25r>, x, y [renderer: GlyphRenderer(id='ef36793e-695d-4e7a-8366-ab4bc90fffd6', ...)]
W-1005 (SNAPPED_TOOLBAR_ANNOTATIONS): Snapped toolbars and annotations on the same side MAY overlap visually: GMapPlot(id='e415b796-703d-4175-82da-5b992ac84734', ...)
E-1001 (BAD_COLUMN_NAME): Glyph refers to nonexistent column name: queries: cyber+securityr>Job levels: [entry_level mid_level senior_level]r